# 使用工具的BabyAGI

这个笔记本在[baby agi](baby_agi.html)的基础上进行了扩展，展示了如何替换执行链。之前的执行链只是一个随意编造的LLM。通过将其替换为一个具有工具访问权限的代理，我们希望能够获得真实可靠的信息。

## 安装和导入所需的模块

In [1]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## 连接到向量存储

根据您使用的向量存储，此步骤可能会有所不同。

In [2]:
# 安装faiss-cpu库
%pip install faiss-cpu > /dev/null
# 安装google-search-results库
%pip install google-search-results > /dev/null

# 导入InMemoryDocstore类
from langchain.docstore import InMemoryDocstore
# 从langchain_community.vectorstores模块中导入FAISS类
from langchain_community.vectorstores import FAISS

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# 定义嵌入模型
embeddings_model = OpenAIEmbeddings()

# 初始化空的向量存储
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## 定义链条

BabyAGI依赖于三个LLM链条：
- 任务创建链条，用于选择要添加到列表中的新任务
- 任务优先级链条，用于重新设置任务的优先级
- 执行链条，用于执行任务


注意：在这个笔记本中，执行链条现在将是一个代理。

In [4]:
# 导入所需的模块
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

# 创建一个用于生成待办事项列表的模板
todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)

# 创建一个基于OpenAI的语言模型链
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)

# 创建一个用于搜索的API包装器
search = SerpAPIWrapper()

# 创建工具列表
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]

# 创建前缀和后缀
prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""

# 创建ZeroShotAgent的提示
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [5]:
# 导入所需的库
from openai import OpenAI
from llm_chain import LLMChain
from zero_shot_agent import ZeroShotAgent
from agent_executor import AgentExecutor

# 创建 OpenAI 模型实例
llm = OpenAI(temperature=0)

# 创建 LLMChain 实例，将 OpenAI 模型和提示作为参数传入
llm_chain = LLMChain(llm=llm, prompt=prompt)

# 获取工具名称列表
tool_names = [tool.name for tool in tools]

# 创建 ZeroShotAgent 实例，将 LLMChain 实例和允许使用的工具名称列表作为参数传入
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

# 使用 AgentExecutor 类从 Agent 和工具列表创建 AgentExecutor 实例
# 设置 verbose 参数为 True，以便在执行过程中打印详细信息
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)


### 运行 BabyAGI

现在是时候创建 BabyAGI 控制器并观察它尝试实现你的目标了。

In [6]:
目标 = "编写今天的旧金山天气报告"

In [7]:
# LLMChains的日志记录
verbose = False
# 如果为None，将一直进行下去
max_iterations: Optional[int] = 3
# 从LLM创建BabyAGI实例
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [9]:
# 调用baby_agi函数，并传入一个字典作为参数
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I need to come up with a todo list
Action: TODO
Action Input: Write a weather report for SF today

1. Research current weather conditions in San Francisco
2. Gather data on temperature, humidity, wind speed, and other relevant weather conditions
3. Analyze data to determine current weather trends
4. Write a brief introduction to the weather report
5. Describe current weather conditions in San Francisco
6. Discuss any upcoming weather changes
7. Summarize the weather report
8. Proofread and edit the report
9. Submit the report I now know the final answer
Final Answer: The todo list for writing a weather report for SF today is: 1. Research current weather conditions in San Francisco; 2. Gather data on temperature, humidity, wind speed, and other relevant weather conditions; 3. Analyze data to determine current weather trends; 4. Write a brief introduction t

{'objective': 'Write a weather report for SF today'}